In [26]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'NSE'

from ib_insync import util, IB, LimitOrder
util.startLoop()

from loguru import logger

# Set the root
from from_root import from_root
ROOT = from_root()

# set vscode path
import sys
sys.path.append(str(ROOT / 'src'))

# To import use 
# from src.<filename> import <function>
from src.utils import (Vars, get_pickle, 
                       quick_pf, place_orders)

_vars = Vars(MARKET)
PORT = port = _vars.PORT

DATAPATH = ROOT / 'data' / MARKET

In [27]:
df_tgt = get_pickle(DATAPATH / "df_naked_targets.pkl")
df = df_tgt.dropna(subset=['expPrice'])

OPT_COLS = _vars.OPT_COLS[0]

In [30]:
# REMOVE PORTFOLIO SYMBOLS FROM TARGET
# ====================================

# Get the portfolio
with IB().connect(port=PORT) as ib:
    df_pf = quick_pf(ib)

# Remove targets which are already in the portfolio
if df_pf:
    df = df[~df.symbol.isin(set(df_pf.symbol))]

In [31]:
# REMOVE OPEN ORDER SYMBOLS
# =========================

with IB().connect(port=PORT) as ib:
    ib.reqAllOpenOrders()  # To kickstart collection of open orders
    # ib.sleep(0.3)
    trades = ib.trades()

if trades:
    all_trades_df = (util.df(t.contract for t in trades).join(
        util.df(t.orderStatus
                for t in trades)).join(util.df(t.order for t in trades),
                                        lsuffix="_"))

    all_trades_df.rename({"lastTradeDateOrContractMonth": "expiry"},
                            axis="columns",
                            inplace=True)
    trades_cols = ["conId", "symbol", "secType", "expiry",
        "strike", "right", "orderId", "permId", "action",
        "totalQuantity", "lmtPrice", "status",]

    dfo = all_trades_df[trades_cols]
    df_openords = dfo[all_trades_df.status.isin(_vars. ACTIVE_STATUS)]

else:
    df_openords = None
    

try:
    if df_openords:  # There are some open orders
        df = df[~df.symbol.isin(set(df_openords.symbol))]
except ValueError:
    pass

In [32]:
# Make (contract, order) tuple
cos = [(contract , LimitOrder('Sell', qty, price))
    for contract, qty, price in zip(df.contract, df.lot_size, df.expPrice)]

In [35]:
dfo

,conId,symbol,secType,expiry,strike,right,orderId,permId,action,totalQuantity,lmtPrice,status
0,671077545,DIXON,OPT,20240125,8350.0,C,155,600508108,SELL,100.0,6730.0,PreSubmitted
1,668774949,TATAMOTOR,OPT,20240125,965.0,C,402,600508355,SELL,1425.0,15522.5,PreSubmitted
2,668775070,WIPRO,OPT,20240125,582.5,C,530,600508483,SELL,1500.0,8082.0,PreSubmitted
3,669204058,SUNTV,OPT,20240229,475.0,P,657,600508610,SELL,1500.0,17622.0,PreSubmitted
4,667885932,BANKNIFTY,OPT,20240103,48300.0,C,16,600507969,SELL,15.0,81.5,Submitted
...,...,...,...,...,...,...,...,...,...,...,...,...
1219,674129095,NIFTY50,OPT,20240201,24000.0,C,0,600507964,SELL,50.0,9674.5,Cancelled
1220,670518204,NIFTY50,OPT,20240111,23000.0,C,0,600507959,SELL,50.0,3360.5,Cancelled
1221,670518198,NIFTY50,OPT,20240111,22950.0,C,0,600507958,SELL,50.0,3397.0,Cancelled
1222,668153261,NIFTY50,OPT,20240104,22150.0,C,0,600507957,SELL,50.0,652.5,Cancelled
